# Projeto 2: Regressão Logística e Redes Neurais

O projeto consiste em explorar técnicas de classificação utilizando o _dataset_ [CINIC-10](https://github.com/BayesWatch/cinic-10), que consiste em um conjunto de 100000 imagens anotadas em 10 classes, que são:

- Airplane
- Automobile
- Bird
- Cat
- Deer
- Dog
- Frog
- Horse
- Ship
- Truck

Para resolver o problema, serão implementadas soluções que utilizam regressão logística multinomial ou rede neural.

In [ ]:
import gc
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from random import seed, shuffle, sample
from datetime import datetime
from copy import deepcopy
from time import time

O _dataset_ a ser utilizado já está separado em conjuntos de treinamento, validação e teste, em arquivos **.npz**. Inicialmente, consideraremos apenas os conjuntos de treinamento e validação. Podemos economizar memória lendo os valores alvo como _int8_.

In [ ]:
train = np.load('Dataset/train.npz')
valid = np.load('Dataset/val.npz')
X, X_v  = train['xs'], valid['xs']
Y, Y_v = train['ys'].astype('int8') , valid['ys'].astype('int8')

# Constantes de classe
classes = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

Com os dados em memória, podemos **normalizá-los** de diversas maneiras. As estatísticas utilizadas para normalização são as mesmas para todos os conjuntos. Esse processo pode consumir bastante memória então, para economia, sempre que possível os tipos serão alterados.

In [ ]:
def get_stats(data, choice=1):
    means,stds,mins,ranges = [],[],[],[]
    
    # Stats for normalization
    if choice == 1 or choice == 3:
        mins = np.apply_along_axis(np.amin, 0, data).astype('int16')
        maxs = np.apply_along_axis(np.amax, 0, data).astype('int16')
        ranges = maxs - mins
    if choice == 2 or choice == 3:
        means = np.apply_along_axis(np.mean, 0, data).astype('float16')
    if choice == 2:
        stds = np.apply_along_axis(np.std, 0, data).astype('float16')
    
    return {'mean':means, 'std':stds, 'mins':mins, 'range':ranges}
    
def normalize_data(data, stats, choice=1):
    ''' Returns the normalized dataset.
    
        Parameters:
            data (array) : numpy array with the dataset.
            stats (array): numpy array with stats given by "get_stats". 0: means. 1: stds. 2:mins. 3:ranges
            choice (int) : integer indicating the transformation to be used.

        Returns:
            data (array list):  transformed data (original data is lost).
    '''

    #### Transforming the dataset ####
    # Min-max normalization
    if choice == 1:
        data = np.apply_along_axis(lambda x: (x - stats['mins'])/stats['range'], 1, data).astype('float32')
            
    # Standardization
    elif choice == 2:
        data = np.apply_along_axis(lambda x: (x - stats['mean'])/stats['std'], 1, data).astype('float32')
            
    # Mean normalization
    elif choice == 3:
        data = np.apply_along_axis(lambda x: (x - stats['mean'])/stats['range'], 1, data).astype('float32')

    return data

Com os dados normalizados, podemos começar a trabalhar com eles. Primeiramente, foi implementada uma **Regressão Logística Multinomial** para tentar solucionar o problema, utilizando a função _softmax_ para múltiplas classes. 

Foi utilizado **Batch Gradient Descent**, sem _early stopping_ e salvando o melhor conjunto de coeficientes.

Para melhor visualização, os custos em cada conjunto por época foram guardados.

In [ ]:
# Activation function
def softmax(x):
    x -= np.max(x, axis=1, keepdims=True)          # Numeric Stability
    x_exp = np.exp(x)
    return x_exp/x_exp.sum(axis=1, keepdims=True)
    
def prob(X, T):
    return softmax(X.dot(T))

def predict(X, T):
    y_scores = X.dot(T)
    return np.argmax(y_scores, axis=1)

# Cost function
def cost(Y, Y_probs):
    correct_probs = Y_probs[np.arange(Y.size), Y]
    return (-1 * log(correct_probs)).mean()

def cost_derivative(X, Y, Y_probs):
    Y_probs[np.arange(Y.size),Y] -= 1
    Y_probs /= Y.size
    return X.T.dot(Y_probs)

def log(x, bound=1e-16):
    return np.log(np.maximum(x,bound))

# Gradient Descent
def gd_step(X, Y, T, Y_prob, alpha):
    return T - alpha * cost_derivative(X, Y, Y_prob)

def gradient_descent(X, Y, X_v, Y_v, T, alpha=0.01, e_lim=300):
    
    # First losses and scores
    Y_prob = prob(X, T)
    Y_v_prob = prob(X_v, T)
    loss = cost(Y, Y_prob)
    best_loss = cost(Y_v, Y_v_prob)

    # History
    best_T = T.copy()
    history = {'cost': [loss], 'v_cost': [best_loss]}
    
    # Descent
    for i in range(e_lim):
        
        # New theta
        T = gd_step(X, Y, T, Y_prob, alpha)
        
        # New scores and losses
        Y_prob = prob(X, T)
        loss = cost(Y, Y_prob)
        
        # Validation
        Y_v_prob = prob(X_v, T)
        v_loss = cost(Y_v, Y_v_prob)
        
        # Updating best loss
        if v_loss < best_loss:
            best_loss = v_loss
            best_T = T.copy()
        
        # History
        history['cost'].append(loss)
        history['v_cost'].append(v_loss)
        print(f"Epoch {i+1:04d}/{e_lim:04d}", f"loss: {loss:.4f} | val loss: {v_loss:.4f}")
    
    print()
    return best_T, history

Com a regressão pronta, basta iniciar os coeficientes para o treinamento. Para isso, foi utilizada a _Xavier Initialization_.

In [ ]:
def init_coefs(features, dim2, rand_seed=None):
    rand = np.random.RandomState(seed=rand_seed)
    return np.sqrt(2/(features + 1)) * rand.randn(features, dim2)

In [ ]:
def run_logistic(X, X_v, Y, Y_v, alpha, n_epochs, n_classes):
    '''
        Runs multinomial logistic regression.
        Uses Xavier Random Initialization of coefficients.
        Gradient descent: softmax function.
    '''
    
    T = init_coefs(X.shape[1], n_classes, 57).astype('float32')

    print("Regression:")
    T, hist = gradient_descent(X, Y, X_v, Y_v, T, alpha, n_epochs)
    return T, hist

Adicionando o _bias_ , coluna adicional no início do conjunto de exemplos, para representar o termo _intercept_ , podemos treinar o modelo.
Após muitos testes, parâmetros bons para a regressão definidos foram _learning rate_ de 0.01 e limite de 300 épocas.

In [ ]:
choice = 2
stats = get_stats(X, choice)

In [ ]:
# Normalization
X = normalize_data(X, stats, choice)
print("Training Data Normalized!")
X_v = normalize_data(X_v, stats, choice)
print("Validation Data Normalized!")

Xb = np.insert(X, 0, 1, axis=1)
X_vb = np.insert(X_v, 0, 1, axis=1)
print("Bias Added")

In [ ]:
T, history = run_logistic(Xb, X_vb, Y, Y_v, 0.01, 300, len(classes))

Após a regressão, pode-se verificar as curvas de aprendizado de treinamento e de validação por meio dos valores salvos.

In [ ]:
def learning_with_history(history):
    '''
        Plots learning curves from history (dictionary of lists)
    '''
    keys = sorted(history.keys())
    for k in keys:
        plt.plot(history[k])
        
    plt.legend(keys, loc='upper left')
    plt.xlabel('Epoch')
    plt.ylabel('Cost')
    plt.title('Learning Curve')
    plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
learning_with_history(history)

Para avaliar o desempenho do modelo, diversas métricas podem ser utilizadas, como:
- Accuracy
- Precision
- Recall
- F1-Score

Porém, para múltiplas classes, apenas _accuracy_ possui definição escalável. As outras métricas estão definidas **por classe**, e a média pode ser utilizada como métrica única se desejado. Um modo fácil de calcular essas métricas é com a **matriz de confusão** da solução.

In [ ]:
# Confusion matrix for the results.
def confusion_matrix(Y, Y_pred, classes):
    conf = np.zeros((classes,classes)).astype('int32')
    for i in range(Y.size):
        conf[Y[i], Y_pred[i]] += 1
    return conf

# Accuracy from confusion matrix. True/total
def accuracy(confusion):
    true_amount = confusion.trace()
    total = confusion.sum()
    return true_amount/total
    
# Precision per class from confusion matrix.
def precision(confusion):
    diag = np.diagonal(confusion)
    return diag/confusion.sum(0)

# Recall per class from confusion matrix.
def recall(confusion):
    diag = np.diagonal(confusion)
    return diag/confusion.sum(1)

# F1 Score per class from precision and recall
def f1_score(prec, rec, bound=1):
    return 2*prec*rec/(prec+rec+bound)
    

# Function to calculate metrics for evaluation
def get_metrics(target, predictions, classes):
    conf = confusion_matrix(target, predictions, classes)
    
    # Metrics
    acc = accuracy(conf)
    prec= precision(conf)
    rec = recall(conf)
    f1 = f1_score(prec, rec)
    avg_acc = (prec + rec)/2
    
    return {'accuracy':acc, 'norm_acc':avg_acc, 'precision':prec, 'recall':rec, 'f1':f1}, conf

A matriz de confusão em si também é interessante de visualizar, por conter muitas informações pertinentes ao desempenho do modelo.

In [ ]:
def plot_confusion_matrix(confusion, classes, model='Neural Network'):
    '''
        Plots an already created confusion matrix for a generic amount of classes.
    '''
    
    fig, ax = plt.subplots(1)

    #Bounding box
    ax.spines['bottom'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['right'].set_color('black')

    plt.title('Confusion Matrix for ' + model)

    #Ticks
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thr = confusion.max()/2
    for i, j in it.product(range(confusion.shape[0]), range(confusion.shape[1])):
        plt.text(j, i, confusion[i, j],
            horizontalalignment='center',
            color='white' if confusion[i, j] > thr else 'black')

    plt.grid(False)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

    plt.imshow(confusion, interpolation='nearest', cmap='Blues')
    plt.show()

    return fig, ax

Com a regressão já feita, podemos testá-la e obter sua matriz de confusão.

In [ ]:
def test_logistic(X, Y, T, classes):
    '''
        Tests logistic regression with implemented metrics.
        Accuracy, normalized accuracy, precision, recall, f1-score.
    '''
    
    pred = predict(X, T)
    met, conf = get_metrics(Y, pred, len(classes)) 
    plot_confusion_matrix(conf, classes, model = 'Multinomial Logistic Regression')
    
    np.set_printoptions(precision=4)
    print(f'Accuracy: {met["accuracy"]:.4f}')
    print(f'Normalized Accuracy: {met["norm_acc"].mean():.4f}')
    print(f'Precision per class: {met["precision"]} (avg. precision: {met["precision"].mean():.4f})')
    print(f'Recall per class: {met["recall"]} (avg. recall: {met["recall"].mean():.4f})')
    print(f'F1-Score per class: {met["f1"]} (avg. f1-score: {met["f1"].mean():.4f})')
    print()

In [ ]:
# Testing
print("Training Metrics:")
test_logistic(Xb, Y, T, classes)
    
print()
print("Validation Metrics:")
test_logistic(X_vb, Y_v, T, classes)

In [ ]:
del Xb, X_vb, T

Como o resultado está longe de satisfatório, devemos perseguir outras técnicas para resolver o problema.

Podemos construir uma **rede neural densa** em seguida, para comparar o desempenho. A rede neural implementada possui três classes:
- Meta, classe que possui metadados do treinamento do modelo, podendo ser utilizada para obter informações para análise.
- Network, classe que representa a rede, podendo ser construída com qualquer quantidade de camadas (maior que 2), com ou sem regularização, e podendo opcionalmente utilizar a função _softmax_ na última camada.
- Optimizer, classe que implementa dois otimizadores para a rede neural: **Adadelta** e **Adam**, que podem ser opcionalmente utilizados.

No momento, exploraremos a rede neural sem otimizadores. A rede implementa _batch_, _stochastic_ e _mini-batch_ _gradient descent_ . Há suporte também para salvar a rede em um arquivo via _NumPy_ ou carregar um arquivo que contém uma rede.

In [ ]:
# Set seed based on current time
seed(datetime.now())

def sigmoid(x):
    '''Function for calculating the sigmoid and preventing overflow

        Parameters:
            x (float): Value for which the sigmoid will be calculated

        Returns:
            float : Sigmoid of x
    '''
    # The masks rounds values preventing np.exp to overflow
    x[x >  50] =  50.0
    x[x < -50] = -50.0
    return 1/(1 + np.exp(-x))

In [ ]:
class Meta:
    def __init__(self, T, m, l, vl, batch_size, sampling=0):
        self.index = 0  # Saves the position in the random list
        self.iters = 0  # Counts the number of weights updates 
        self.bound = m  # Total amount of samples
        self.best_loss = 0
        self.best_T = T.copy()
        self.epochs_count = 0      # Amount of completed epochs
        self.start_time = time()   # Training start time
        self.sampling = sampling   # Number of iterations which samples are collected
        self.samples_list = list(range(m)) # Radomized samples for stochastic methods
        self.history = {'loss':[l], 'v_loss':[vl]}
        shuffle(self.samples_list)
        
        # Checks it the batch is an integer (n samples) or percentage and, if 
        # it's a percentage, adjust to number of samples
        if not isinstance(batch_size, int) :
            self.batch_size = int(np.ceil(m*batch_size))
        else : 
            self.batch_size = batch_size
        
        # If analysis data will be kept, saves time and thetas
        if (self.sampling):
            self.coef = [deepcopy(T)] # Keeps trained coeficients per epoch
            self.time = [0.0]   # Marks when a ecpoch was complete

    def update(self, cost, samples_sets, T, e_lim):
        '''Updates hyperparameters (epoch count, samples ramdomization)
        
            Parameters:
                T (list of np.ndarray) : Coeficients from the current epoch
                analysis (bool) : If true, keeps arguments for analysis
            Returns:
                change (bool) : if true, epoch finished
        '''
        
        # Update index (add samples used in iteration)
        self.iters += 1
        self.index += self.batch_size
        change = False
        
        # If epoch completed
        if self.index >= self.bound :
            self.index = 0             # Reset samples index
            self.epochs_count += 1     # Count finished epoch
            shuffle(self.samples_list) # Reshuffle samples
            (X,Y,Xv,Yv) = samples_sets
            loss = cost(X,Y)
            v_loss = cost(Xv,Yv)
            self.update_history(loss, v_loss)
            
            # Updates best thetas
            if self.best_loss > v_loss:
                self.best_loss = v_loss
                self.best_T = self.theta.copy()
            print(f"Epoch {self.epochs_count:04d}/{e_lim:04d}", f"loss: {loss:.4f} | val loss: {v_loss:.4f}")

        # Data for further analysis (Consumes time and memory)
        if (self.sampling and self.iters//self.sampling) :
            self.iters = 0
            self.time.append(time() - self.start_time) # Adds time until epoch is done
            self.coef.append(deepcopy(T)) # Adds current epoch cost
            
        return change

    def update_history(self, loss, v_loss):
        self.history['loss'].append(loss)
        self.history['v_loss'].append(v_loss)
        
    def get_batch(self):
        '''Get samples indexes for the next batch'''
        return self.samples_list[self.index : self.index+self.batch_size]

    def __str__(self):
        out = f'<Meta Object at {hex(id(self))}>'
        out += f'Samples per Epoch: {self.bound}'
        out += f'Current samples index: {self.index}'
        out += f'Batch size: {self.batch_size}'
        out += f'Epochs complete so far: {self.epochs_count}'
        return out

In [ ]:
class Network:
    def __init__(self, model, f='sg', reg_lambda=0, T=0, seed=0):
        '''Initializes coeficients tables with the network weights
        
        Parameters: 
            model (list) : List with the amount of nodes per layer (without bias)
            f (String) : Identification for the function to be minimized
            reg_lambda (float) : Regularization parameter for the network (0 disables regularization)
            T (list of numpy.ndarray): If instanciated, uses T as initial thetas
        '''
        self.reg_lambda = reg_lambda
        self.f = f
        self.theta = []

        # Generates a random seed based on current time
        if not seed : int(divmod(time(), 1)[1])

        # If no preset, instanciate thetas and set random initial thetas
        for (n,m) in zip(model[1:],model[:-1]): 
            # Instanciate weights with Xavier initialization
            rand = np.random.RandomState(seed=seed)
            self.theta.append((np.sqrt(2/(m+1)) * rand.randn(n,m+1)).astype(np.float32))

    def cost(self, X, Y):
        '''Calculates the current cost for the given samples (features and outputs)

        Parameters:
            X (numpy.ndarray): NxM matrix with N features and M samples
            Y (numpy.ndarray): KxM matrix with K output nodes and M samples

        Returns:
            float : Total cost for the current network and the given samples 
        '''
        reg = 0 # Regularization value (weight reduction)
        e = 10**-6 # Offset used to avoid log(0) (prevents NaNs)
        m = Y.shape[1]  # Get amount of samples
        fun = lambda x : (x[:,1:]*x[:,1:]).sum() # Sum squared parameters without bias 
        H = self.frag_forward(X, 10) # Get output layer activation values

        # Calculate cost function
        if self.f == 'sg': # Use sigmoid cost
            cost = -(Y*np.log(H+e) + (1-Y)*np.log((1+e)-H)).sum()/m
        elif self.f == 'sm': # Use softmax cost
            cost_mat = softmax(H)
            cost = (-Y * np.log(cost_mat+e)).sum(axis=0).mean()

        # Calculate regularization, if parameter is set
        if self.reg_lambda : reg = self.reg_lambda*(sum(map(fun, self.theta))/(2*m))

        return cost + reg

    def cost_deriv(self, H, Y):
        '''Calculates the current cost for the given samples (features and outputs)

        Parameters:
            H (numpy.ndarray): NxM matrix with output layer activation values (N node X M samples)
            Y (numpy.ndarray): KxM matrix with K output nodes and M samples

        Returns:
            float : Total cost for the current network and the given samples 
        '''
        m = Y.shape[1]  # Get amount of samples

        if self.f =='sg': # Use sigmoid derivative
            return H - Y
        elif self.f == 'sm': # Use softmax derivative
            return softmax(H) - Y

    def forward(self, features, nodes=0):
        '''Execute the forward propagation using the defined thetas
        
        Parameters: 
            features (numpy.ndarray) : Column vector with input features (without bias)
            nodes (list) : if instanciated, saves the nodes activation values for every layer

        Returns:
            numpy.ndarray : Array with the propagated value for the output layer
        '''
        m = features.shape[1] # Get amount of samples to be propagated

        for table in self.theta :
            features = np.insert(features, 0, 1, axis=0)
            if isinstance(nodes, list) : nodes += [features] 
            features = sigmoid(table.dot(features))

        if isinstance(nodes, list) : nodes += [features]
        return features

    def backprop(self, X, Y):
        '''Execute gradient calculation for the given thetas and samples
        
        Parameters: 
            X (numpy.ndarray) : NxM matrix with M samples and each sample with N features
            Y (numpy.ndarray) : KxM matrix with M samples ana each samples with K output nodes

        Returns:
            list of numpy.ndarray : Gradients for each set of thetas in the network 
        '''
        reg_lambda = self.reg_lambda # Regularization parameter
        theta = self.theta # Alias for the parameters
        m = Y.shape[1] # Amount of samples to be backpropagated

        layer = [] # For keeping the activation values
        grad = [np.zeros(i.shape) for i in theta] # For keeping the partial derivatives
        H = self.forward(X, nodes=layer) # Calculate hypotesis for every output node of every sample
        sigma = [np.zeros(i.shape) for i in layer[1:]] # For keeping the activation errors (except input layer)
        reg = 0

        sigma[-1] = self.cost_deriv(H, Y)
        
        # Back propagate error to hidden layers (does not propagate to bias nodes)
        for i in reversed(range(1, len(sigma))):
            sig_d = layer[i][1:,:]*(1-layer[i][1:,:]) # Remove bias from layers for backpropagation
            sigma[i-1] = (theta[i][:,1:].T).dot(sigma[i])*sig_d # Remove bias from thetas as well

        # Accumulate derivatives values for every theta (does not update thetas)
        # - Biases are not regularized, so the bias weights are casted to zero
        for i in range(len(grad)):
            if reg_lambda : reg = np.insert(theta[i][:,1:]*reg_lambda, 0, 0, axis=1)
            grad[i] = (grad[i] + sigma[i].dot(layer[i].T))/m + reg
        
        return grad

    def train(self, X, Y, Xv, Yv, type='m', opt=None, t_lim=7000, e_lim=100000, rate=0.01, mb_size=32, sampling=0, betas=(0,0)):
        '''Trains the model until one of the given limits are reached

        Parameters:
            X (Float 2dArray): The coeficient matrix.
            Y (Float 2dArray): The results matrix.
            type (int): The choice of descent ('s'-stoch|'m'-mini|'b'-batch)
            opt (String): Selects the optimizer for the decent (None|adadelta|adam)

        Returns:
            Meta : Class containing the runtime info.
        '''
        # Initializes epochs metadata class
        batch_size = {'b':Y.shape[1],'m':mb_size,'s':1}.get(type) # Get number of samples
        data = Meta(self.theta, Y.shape[1], self.cost(X,Y), self.cost(Xv,Yv), batch_size, sampling=sampling) # Saves hyperparameters and other info for analisys 
        optmizer = Optimizer(rate, choice=opt, T=self.theta, batch=mb_size, beta=betas)

        # Starting descent
        while (time() - data.start_time) <= t_lim:
            # Getting new Thetas
            b = data.get_batch() # Get indexes for mini batch
            grad = self.backprop(X[:,b], Y[:,b])    
            delta = optmizer.optimize(grad)
            
            # Update coefficients
            for i in range(len(delta)):
                self.theta[i] += delta[i]

            data.update(self.cost, (X,Y,Xv,Yv), self.theta, e_lim)
            
            # Check termination
            if data.epochs_count >= e_lim : 
                print("NOTE: Epochs limit for descent reached.")  
                self.theta = data.best_T     
                return data
            
        print("NOTE: Time limit for descent exceded.")
        self.theta = data.best_T
        return data

    def frag_forward(self, X, parts):
        '''Wrapper for the forward propagation which splits the M samples in slices
           Prevents numpy memory spikes during large matrix multiplications           

        Parameters:
            X (Float 2dArray): NxM matrix with N input feratures and M samples
            type (int): The choice of descent ('s'-stoch|'m'-mini|'b'-batch).

        Returns:
            numpy.ndarray : Array with the propagated value for the output layer
        '''
        m = X.shape[1]
        size = int(np.ceil(m/parts)) # Get size of each batch
        out_layer = np.zeros((self.theta[-1].shape[0],m)) # Prealocate output layer
        batches = [i*size for i in range(int(np.ceil(parts)+1))] # Slices indexes
        for (s,e) in zip(batches[:-1], batches[1:]): # Propagate for each slice
            out_layer[:,s:e] += self.forward(X[:,s:e])
        return out_layer

    def accuracy(self, X, Y, T=0):
        '''Caculates the cost for a set of samples in the current network
        
            Parameters:
                X (Float 2dArray): NxM matrix with N input features and M samples
                Y (Int 2dArray): NxM matrix with the expected M output layers
            
            Returns:
                float : Percentage of correct predictions for the M samples
        '''
        m = Y.shape[1] 
        H = self.frag_forward(X, 10)
        H = H.argmax(axis=0)
        Y = Y.argmax(axis=0)
        hits = (H==Y).sum()
        return hits*100/m
    
    def predict(self, X):
        '''
           Predicts classes of examples.
           
           Parameters:
                X (Float 2dArray): NxM matrix with N input features and M samples
                
           Returns:
                Int Array: N-dimensional array with predictions
        '''
        
        H = self.frag_forward(X, 10)
        H = H.argmax(axis=0)
        return H

    def save(self, file_name):
        '''Function for saving the current network to a file'''
        save_list = [np.array([self.reg_lambda,self.f])]
        save_list += self.theta
        np.savez(file_name, save_list)
        print(f"Model saved as {file_name}")
 
    def load(self, file_name):
        '''Function for loading a Network from a file'''
        obj = np.load(file_name)
        self.reg_lambda = int(obj['arr_0'][0][0])
        self.f = str(obj['arr_0'][0][1])
        self.theta = []
        for T in obj['arr_0'][1:]:
            self.theta.append(T)
        print(f"Model {file_name} loaded")

    def __str__(self):
        funcs = {'sg':'Sigmoid', 'sm':'Softmax'}
        out = f'<Network Object at {hex(id(self))}>\n'
        out += f'Composed of {len(self.theta)+1} layers:\n'
        for i,n in enumerate(self.theta):
            out += f'   layer {i+1} - {n.shape[1]} nodes\n'
        out += f'   Out layer - {self.theta[-1].shape[0]} nodes\n'
        out += f'Cost function: {funcs[self.f]}\n'
        out += f'Regularization parameter: {self.reg_lambda}\n'
        out += f'Amount of weights: {sum([x.size for x in self.theta])}\n'
        return out

In [ ]:
class Optimizer:
    def __init__(self, rate, choice=None, T=0, batch=0, beta=(0,0)):
        self.choice = choice
    
        if choice=='adadelta':
            self.e = 10**-8
            self.decay = 0.99
            self.batch = batch 
            self.avg = [np.zeros(t.shape) for t in T]
            self.delta = [np.zeros(t.shape) for t in T]
        elif choice=='adam':
            self.e = 1e-8
            self.rate = rate
            self.t = 0
            self.beta = beta
            self.mt = [np.zeros(t.shape) for t in T]
            self.vt = [np.zeros(t.shape) for t in T]
            self.batch = batch
        else:
            self.rate = rate
        
    def optimize(self, grad):
        
        if self.choice=='adadelta':
            delta = self.adadelta(grad)
        elif self.choice=='adam':
            delta = self.adam(grad)
        else: # Vanilla
            delta = [-self.rate*g for g in grad]
        return delta

    def adadelta(self, grad):
        eps = self.e
        decay = self.decay
        batch = self.batch
        new_deltas = []
        for i,(g,avg,delta) in enumerate(zip(grad, self.avg, self.delta)):
            # Calculate new optimized delta
            avg = decay*avg + (1-decay)*np.square(g)
            new_deltas.append(-(np.sqrt(delta+eps)/np.sqrt(avg+eps))*g)
            # Updates for next iteration
            self.avg[i] = avg
            self.delta[i] = decay*delta + (1-decay)*np.square(new_deltas[-1])

        return new_deltas
    
    def adam(self, grad):
        self.t+=1
        mt = self.mt
        vt = self.vt
        new_deltas = []
        
        for i,g in enumerate(grad):
        
            # Calculating moving averages
            mt[i] = self.beta[0]*mt[i] + (1-self.beta[0])*g
            vt[i] = self.beta[1]*vt[i] + (1-self.beta[1])*g*g
            
            # Bias-corrected estimates for moment
            mt_b = mt[i]/(1-(self.beta[0]**self.t))
            vt_b = vt[i]/(1-(self.beta[1]**self.t))
            
            delta = -1 * (self.rate * mt_b)/(np.sqrt(vt_b)+self.e)
            new_deltas.append(delta)
        
        # Update
        self.mt = mt
        self.vt = vt
        
        return new_deltas

Com algumas adaptações, as funções de teste podem ser as mesmas. Porém, a entrada necessita de pré-processamento, pois ao contrário da regressão logística, a rede neural implementada utiliza _One-Hot Encoding_ na camada de saída.

O método _predict_ da classe _Network_ retorna a saída ao formato original, para avaliação.

In [ ]:
def out_layers(array):
    '''Converts values from a list to one hot enconded output layers

        Parameters:
            array (list of ints): Contains the integer ID for each class
    
        Returns:
            np.ndarray : 2D array where each colum is a output layer
    '''
    lower = min(array)
    upper = max(array)
    lines = upper-lower+1
    new_arr = np.zeros((lines,len(array)), dtype=np.int8)

    for j,i in enumerate(map(lambda x : x-lower, array)):
       new_arr[i,j] = 1

    return new_arr

Podemos fazer um _wrapper_ para criar a rede, com uma arquitetura testada anteriormente que forneceu bons resultados.
O wrapper também transpõe os dados de entrada, já que este, diferentemente da regressão logística, considera as linhas sendo Features e as colunas como Samples.

In [ ]:
def create_neural_network(X, Y, Yv, reg=0, f='sg'):
    '''
        Creates fully-connected neural network.
        Parameters:
            X,Xv: MxN matrix with N input features and M samples
            Y,Yv: M-dimensional array with target values.
            reg : regularization parameter
            f   : activation function of choice (sg: sigmoid | sm: softmax)
    '''
    
    # Adjusting input matrices (assumes X is normalized)
    Yn = out_layers(Y)
    Yvn=out_layers(Yv)
    print("Handled input")
    
    # Builds network object
    feat, out = X.T.shape[0], Yn.shape[0]
    h1, h2 = 256, 128
    arc = [feat, h1, h2, out]
    model = Network(arc, f=f, seed=23, reg_lambda=reg)
    print('Created model')
    print(f"Initial Accuracy: {(model.accuracy(X.T, Yn)/100):.4f}")
    
    return Yn, Yvn, model


Com isso, temos o que precisamos para treinar a rede, que utilizará **Mini-Batch Gradient Descent**.
Após diversos testes, bons parâmetros encontrados consistem em utilizar batches de tamanho 256, e _learning rate_ de 0.01. A função _softmax_ não foi utilizada pois seu resultado é pior. Os outros parâmetros foram escolhidos arbitrariamente.

In [ ]:
Yn, Yvn, model = create_neural_network(X, Y, Y_v)
data = model.train(X.T, Yn, X_v.T, Yvn, type='m', mb_size=256, e_lim=500, t_lim=300, rate=0.01)

Os testes são feitos de maneira similar à regressão logística.

In [ ]:
def test_neural_network(X, Y, model, classes):
    '''
        Function to test neural network.
    '''
    pred = model.predict(X)
    met, conf = get_metrics(Y, pred, len(classes)) 
    plot_confusion_matrix(conf, classes, model='Neural Network')
    
    np.set_printoptions(precision=4)
    print(f'Accuracy: {met["accuracy"]:.4f}')
    print(f'Normalized Accuracy: {met["norm_acc"].mean():.4f}')
    print(f'Precision per class: {met["precision"]} (avg. precision: {met["precision"].mean():.4f})')
    print(f'Recall per class: {met["recall"]} (avg. recall: {met["recall"].mean():.4f})')
    print(f'F1-Score per class: {met["f1"]} (avg. f1-score: {met["f1"].mean():.4f})')
    print()

In [ ]:
print("Training Metrics:")
test_neural_network(X.T, Y, model, classes)
    
print("Validation Metrics:")
test_neural_network(X_v.T, Y_v, model, classes)

Pode-se ver que o resultado é melhor do que o visto para a regressão logística, porém ainda pode melhorar.
Podemos utilizar os otimizadores implementados, e comparar os resultados.

Nos testes com otimizadores, os hiperparâmetros mudaram. Um batch size de 1024, por exemplo, não funciona bem para a rede original, mas funciona melhor que o tamanho original para os otimizadores.

In [ ]:
def optimization_cost(X, Y, Xv, Yv, m=80000):
    plot_info = []

    samples = sample(range(Y.shape[1]), m)
    X = X[:,samples]
    Y = Y[:,samples]
    feat = X.shape[0]
    out = Y.shape[0]

    # Set constant hyperparameters
    e=50
    t=200
    r=0.001
    b=1024
    
    # Variable aspects/hyperparams per plot
    opt = ['adam','adadelta', None]
    title = ['Adam', 'Adadelta', 'Vanilla']
    arc = [3072, 256, 128, 10]

    print("Architecture:", arc)
    for i in range(len(title)):
        model = Network(arc, seed=23)
        data = model.train(X, Y, Xv, Yv,
                            opt=opt[i], 
                            type='m', 
                            t_lim=t, 
                            e_lim=e, 
                            rate=r, 
                            mb_size=b, 
                            betas=(0.9,0.999))
        plot_info.append((title[i], range(0,data.epochs_count+1), data.history['loss']))
        plot_info.append((title[i]+'(Validation)', range(0,data.epochs_count+1), data.history['v_loss']))
        print(title[i]+':', model.accuracy(Xv,Yv), '%')


    for (l,x,y) in plot_info : plt.plot(x, y, label=l)
    plt.title(f"Learning Curve \n {m} samples | {t} sec")
    plt.xlabel("Epochs")
    plt.ylabel("Cost")
    plt.legend()
    plt.show()


In [ ]:
del model, data
gc.collect()
optimization_cost(X.T, Yn, X_v.T, Yvn)

É perceptível no resultado das comparações que o modelo com otimizador Adam sofreu _overfit_ , e por isso o custo de validação volta a subir no final. Esse modelo também é o que gerou melhores resultados nesse teste, então será considerado o melhor.

Para impedir _overfitting_ , no treinamento final será utilizado um fator de regularização de 0.002, assim deixando a convergência mais lenta, então o tempo limite foi aumentado.

In [ ]:
Yn, Yvn, model = create_neural_network(X, Y, Y_v, reg=0.002)
data = model.train(X.T, Yn, X_v.T, Yvn, type='m', opt='adam', mb_size=1024, e_lim=500, t_lim=900, rate=0.001, betas=(0.9,0.999))

In [ ]:
print("Training Metrics:")
test_neural_network(X.T, Y, model, classes)
    
print("Validation Metrics:")
test_neural_network(X_v.T, Y_v, model, classes)

Como esse modelo foi escolhido como o melhor, podemos verificar o **conjunto de teste** nele.

In [ ]:
del X, Y, Yn, Yvn, X_v, Y_v, train, valid
gc.collect()

test = np.load('Dataset/test.npz')
X_t, Y_t = test['xs'], test['ys'].astype('int8')
X_t = normalize_data(X_t, stats, choice)
print("Test Data Normalized!")

In [ ]:
print("Test Metrics:")
test_neural_network(X_t.T, Y_t, model, classes)